In [ ]:
import tensorflow as tf
import datetime, time
import tweepy as tw
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.preprocessing.sequence import pad_sequences

In [ ]:
consumer_key= 
consumer_secret= 
access_token= 
access_token_secret= 

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
def get_tweet(api, username):
    page = 1
    deadend = False
    results = set()
    while True:
        tweets = api.user_timeline(username, page = page)

        for tweet in tweets:
            if (datetime.datetime.now() - tweet.created_at).days < 1:
                if tweet.text.startswith("RT") == True:
                    pass
                else:
                    results.add(tweet.text)
            else:
                deadend = True
                break
        if deadend:
            return results
        if not deadend:
            page+=1
            time.sleep(1)

In [ ]:
tweet_potus = get_tweet(api, "potus")

In [ ]:
tweet_dt = get_tweet(api, "realDonaldTrump")

In [ ]:
all_tweets = tweet_potus.union(tweet_dt)

In [ ]:
tweet = np.asarray([" ".join(list(all_tweets))])

In [ ]:
tr = tf.keras.models.load_model('model.pb')

In [ ]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(tweet)
sequences = tokenizer.texts_to_sequences(tweet)

In [ ]:
data = pad_sequences(sequences, maxlen=1000)

In [ ]:
predict = tr.predict(data)

In [ ]:
print("Prediction: %.2f" % (float(predict[0]/100)))